<a href="https://colab.research.google.com/github/FairyFox5700/AI_Sound_Processing/blob/main/Wav_data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)

from scipy.io import wavfile
import numpy as np
from sklearn.preprocessing import LabelBinarizer
labelbinarizer = LabelBinarizer()
import os
from scipy import signal
from shutil import copyfile
import random

In [16]:
def Convert_To_06(samples):
    sample_rate=8000
    WindowSize = 40
    LeftSeek = 300
    RezFileLength = 0.6


    if  len(samples) <= int(0.6*sample_rate):
        return samples

    Samples_SUM = np.arange(len(samples), dtype='float64')
    ResultS = np.arange(int(RezFileLength * sample_rate), dtype=samples.dtype)
    Samples_Float = np.arange(len(samples), dtype='float64')

    # ------ARRAY -----AverageSum--------------------------
    Average = 0.0
    for i in range(len(samples)):
        Average += samples[i]

    Average = Average / len(samples)
    for i in range(len(samples)):
        Samples_Float[i] = abs(samples[i] - Average)

    MaxSum = 0
    TotalSum = 0
    for i in range(len(Samples_SUM)):
        Samples_SUM[i] = 0
    i = WindowSize
    while i < (len(samples) - WindowSize - 1):
        CurentSum = 0
        J = -WindowSize
        while J < WindowSize:
            CurentSum = CurentSum + Samples_Float[i + J]
            J = J + 1
        Samples_SUM[i] = CurentSum / (2 * WindowSize + 1)
        if MaxSum < Samples_SUM[i]:
            MaxSum = Samples_SUM[i]

        i += 1
    Thresholt = MaxSum / 6
    for asum in Samples_SUM:
        if Thresholt < asum:
            TotalSum += asum
    CenterIndex = 0
    CurentSum = Samples_SUM[0]

    while CurentSum < TotalSum / 2:
        CenterIndex += 1
        CurentSum += Samples_SUM[CenterIndex]

    StartIndex = 0
    BitRateCount = 0
    for i in range(len(Samples_SUM)):
        if Samples_SUM[i] > Thresholt:
            BitRateCount += 1
        else:
            BitRateCount = 0
        if BitRateCount > WindowSize * 4:
            if i > WindowSize * 4 + LeftSeek:
                StartIndex = int(i - (WindowSize * 4 + LeftSeek))
            break


    if CenterIndex < int(StartIndex + (sample_rate * RezFileLength / 2)) and (sample_rate * RezFileLength / 2) < CenterIndex:
        StartIndex = CenterIndex - (sample_rate * RezFileLength / 2)
    if StartIndex + sample_rate * RezFileLength > len(samples):
        StartIndex = len(samples) - sample_rate * RezFileLength
    StartIndex = int(StartIndex)
    for i in range(int(RezFileLength * sample_rate)):
        ResultS[i] = samples[i + StartIndex]
    return  ResultS

def ConvertTo_8K(SourceDir, TargetDirectory,Prefics, ClassType ):
    #  -------Load Files----------------------------------------
   
    Wav_Files = []
    for d, dirs, files in os.walk(SourceDir):
     
        for file in files:
            if file.endswith(".wav"):
                Wav_Files.append(file)
    #  -------Convert to 8K and Raname and Save Files----------------------------------------
    for wfile in Wav_Files:
        curerentFile = SourceDir + "/" + wfile
        
        sample_rate, samples = wavfile.read(curerentFile)

        i = wfile.find(".wav")
        FileName = wfile[:i]
        FileName = FileName.replace("_nohash_", "")

        if sample_rate == 16000:
            lennewarray = int(len(samples) / 2)
            f = signal.resample(samples, lennewarray)
            Samples_1 = np.arange(lennewarray, dtype=samples.dtype)
            for i in range(lennewarray):
                Samples_1[i] = int(f[i])
            curerentFile = TargetDirectory+ "/"+ Prefics + FileName + "_0000_" + ClassType + ".wav"
            print(curerentFile)
            Samples_final = Convert_To_06(Samples_1)
            wavfile.write(curerentFile, 8000, Samples_final)

        if sample_rate == 8000:
            curerentFile = TargetDirectory + FileName + "+0000_" + ClassType + ".wav"
            Samples_final = Convert_To_06(samples)
            wavfile.write(curerentFile, sample_rate, Samples_final)
            
def Reduce_TrainClass(TrainDirectory, TrainPercent):
    Clas3_Files = []
    
    # read 3 class files
    for d, dirs, files in os.walk(TrainDirectory):
        for file in files:
            if file.endswith(".wav"):
                Clas3_Files.append(file)
                
    TrainSize = len(Clas3_Files)           
    ExpectedLength = int(TrainSize * TrainPercent)
    # random index of file to delete
    while TrainSize > ExpectedLength:
        Index = random.randint(0, TrainSize - 1)
        ItemToDelete = Clas3_Files[Index]
        os.remove(TrainDirectory+ "/"+ ItemToDelete)
        print(ItemToDelete)
        Clas3_Files.remove(ItemToDelete)
        print(TrainSize)
        TrainSize-=1
    del Clas3_Files
    
def Divide_TrainTestValid(SourceDirectory, TrainDirectory, TestDirectory, ValidDirectory,TestPercent, ValidPercent):

    Clas1_Files = []
    Clas2_Files = []
    Clas3_Files = []
    Test_Files = []
    Clas1_ID = "cl_1"
    Clas2_ID = "cl_2"
    Clas3_ID = "cl_3"

    Wav_Files = []
    for d, dirs, files in os.walk(SourceDirectory):
        for file in files:
            if file.endswith(".wav"):
                Wav_Files.append(file)
    for fwav in Wav_Files:
        if Clas1_ID in fwav:
            Clas1_Files.append(fwav)
        if Clas2_ID in fwav:
            Clas2_Files.append(fwav)
        if Clas3_ID in fwav:
            Clas3_Files.append(fwav)
  
    TestSize = len(Clas1_Files)
    if TestSize > len(Clas2_Files):
        TestSize = len(Clas2_Files)
    if TestSize > len(Clas3_Files):
        TestSize = len(Clas3_Files)
    ValidSize = int(TestSize * ValidPercent)
    TestSize = int(TestSize * TestPercent)

    # ------------------TEST----------------------------------------

    for i in range(TestSize):
        Index = random.randint(0, len(Clas1_Files) - 1)
        Test_Files.append(Clas1_Files[Index])
        del Clas1_Files[Index]
        Index = random.randint(0, len(Clas2_Files) - 1)
        Test_Files.append(Clas2_Files[Index])
        del Clas2_Files[Index]
        Index = random.randint(0, len(Clas3_Files) - 1)
        Test_Files.append(Clas3_Files[Index])
        del Clas3_Files[Index]

    for tFile in Test_Files:
        src = SourceDirectory + "/"+ tFile
        dst = TestDirectory + "/"+ tFile
        copyfile(src, dst)
   # ------------------TEST---------------------------------------


  # ------------------Valid----------------------------------------
    Test_Files = []
    for i in range(ValidSize):
        Index = random.randint(0, len(Clas1_Files) - 1)
        Test_Files.append(Clas1_Files[Index])
        del Clas1_Files[Index]
        Index = random.randint(0, len(Clas2_Files) - 1)
        Test_Files.append(Clas2_Files[Index])
        del Clas2_Files[Index]
        Index = random.randint(0, len(Clas3_Files) - 1)
        Test_Files.append(Clas3_Files[Index])
        del Clas3_Files[Index]

    for tFile in Test_Files:
        src = SourceDirectory + "/"+ tFile
        dst =ValidDirectory + "/"+ tFile
        copyfile(src, dst)
    # ------------------Valid---------------------------------------


    Train_Files = Clas1_Files + Clas2_Files + Clas3_Files
    for tFile in Train_Files:
        src = SourceDirectory + "/"+  tFile
        dst = TrainDirectory + "/"+  tFile
        copyfile(src, dst)

In [22]:
ConvertTo_8K(SourceDir=r'/content/sample_data/mpi2/go', TargetDirectory=r'/content/sample_data/training2/data', Prefics='go',
                 ClassType='cl_1')
ConvertTo_8K(SourceDir=r'/content/sample_data/mpi2/off', TargetDirectory=r'/content/sample_data/training2/data', Prefics='off',
                 ClassType='cl_2')




/content/sample_data/mpi2/go/go_M`iachDmytro_masc_fast_loud_happy.wav
/content/sample_data/mpi2/go/go_StepaniukIryna_fem_fast.wav
/content/sample_data/mpi2/go/go_ByshovetsNataliia_fem_sad.wav
/content/sample_data/mpi2/go/go_HorobiukVictor_masc_fast.wav
/content/sample_data/mpi2/go/go_SysakIgor_man_fast_sad.wav
/content/sample_data/mpi2/go/go_SaakianNatan_masc_slow_quiet_happy.wav
/content/sample_data/mpi2/go/go_OhorodnikBohdan_masc_slow.wav
/content/sample_data/mpi2/go/go_SysakIgor_man_sad.wav
/content/sample_data/mpi2/go/go_NovikovMykhailo_masc_fast_happy.wav
/content/sample_data/mpi2/go/go_HorobiukVictor_masc_super_fast.wav
/content/sample_data/mpi2/go/go_ElMansourySimon_masc_fast_loud_sad.wav
/content/sample_data/mpi2/go/go_BershadskyiAndrii_masc_fast_loud_happy.wav
/content/sample_data/mpi2/go/go_DolidVolodymyr_masc_quiet.wav
/content/sample_data/mpi2/go/go_LutsenkoOleksandr_masc_fast_happy.wav
/content/sample_data/mpi2/go/go_ByshovetsNataliia_fem_slow_happy.wav
/content/sample_dat

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:86: WavFileWarning: Reached EOF prematurely; finished at 434220 bytes, expected 434264 bytes from header.


/content/sample_data/mpi2/off/off_LebedenkoAnna_fem_happy.wav
/content/sample_data/mpi2/off/off_YanchukSerhii_masc_fast_quiet.wav
/content/sample_data/mpi2/off/off_StepaniukIryna_fem_fast.wav
/content/sample_data/mpi2/off/off_BorodaiKateryna_fem_fast_quiet_sad.wav
/content/sample_data/mpi2/off/off_AidymyrovaNadiia_fem_fast.wav
/content/sample_data/mpi2/off/off_childVoice.wav
/content/sample_data/mpi2/off/off_AidymyrovaNadiia_fem_happy.wav
/content/sample_data/mpi2/off/off_LapaAnastasiia_fem_slow.wav
/content/sample_data/mpi2/off/off_OhorodnikBohdan_masc_fast.wav
/content/sample_data/mpi2/off/off_SysakIgor_man_loud.wav
/content/sample_data/mpi2/off/off_ShenheliiaVolodymyr_masc_fast_quiet.wav
/content/sample_data/mpi2/off/off_M`iachDmytro_masc_slow_loud.wav
/content/sample_data/mpi2/off/off_ShenheliiaVolodymyr_masc_fast_quiet_happy.wav
/content/sample_data/mpi2/off/off_BorodaiKateryna_fem_slow_loud_happy.wav
/content/sample_data/mpi2/off/off_OhorodnikBohdan_masc_fast_loud.wav
/content/sa

In [24]:
Divide_TrainTestValid(SourceDirectory=r'/content/sample_data/training2/data',
                          TrainDirectory=r'/content/sample_data/training2/train',
                          TestDirectory=r'/content/sample_data/training2/test', 
                          ValidDirectory=r'/content/sample_data/training2/valid', 
                          TestPercent=0.1,
                          ValidPercent=0.1)

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
Reduce_TrainClass(TrainDirectory=r'C:\Users\Iryna_Tyshchenko\training\train', TrainPercent =0.1)

other305776dd2_0000_cl_3.wav
8728
go5628d7b72_0000_cl_1.wav
8727
otherb7a0754f0_0000_cl_3.wav
8726
off026290a70_0000_cl_2.wav
8725
off587f32710_0000_cl_2.wav
8724
otherec7d11510_0000_cl_3.wav
8723
goaa2336540_0000_cl_1.wav
8722
other274c008f3_0000_cl_3.wav
8721
other6d1dcca60_0000_cl_3.wav
8720
off50a8ef271_0000_cl_2.wav
8719
otherc6ee87a74_0000_cl_3.wav
8718
godbaf8fc60_0000_cl_1.wav
8717
off4620dc142_0000_cl_2.wav
8716
otherd9b8fab20_0000_cl_3.wav
8715
go1b4c9b894_0000_cl_1.wav
8714
go3e3fa7fd1_0000_cl_1.wav
8713
go6adb8ad90_0000_cl_1.wav
8712
other748cb3081_0000_cl_3.wav
8711
gocab100c90_0000_cl_1.wav
8710
offf17be97f0_0000_cl_2.wav
8709
otherf8f60f592_0000_cl_3.wav
8708
other743edf9d0_0000_cl_3.wav
8707
otherc7b4049e0_0000_cl_3.wav
8706
other8549f25d0_0000_cl_3.wav
8705
offad63d93c2_0000_cl_2.wav
8704
other8f0d3c271_0000_cl_3.wav
8703
offcb2929ce6_0000_cl_2.wav
8702
offc120e80e5_0000_cl_2.wav
8701
other3ffa34571_0000_cl_3.wav
8700
other6e74c5820_0000_cl_3.wav
8699
gobf8d56170_0000_

other68dd409e1_0000_cl_3.wav
8273
other305776dd1_0000_cl_3.wav
8272
othere102119e3_0000_cl_3.wav
8271
goa0f939431_0000_cl_1.wav
8270
gobf90a57a1_0000_cl_1.wav
8269
off5ba724a70_0000_cl_2.wav
8268
other84bf12ff3_0000_cl_3.wav
8267
other11a022ba0_0000_cl_3.wav
8266
otherb7a6f7092_0000_cl_3.wav
8265
go24b821920_0000_cl_1.wav
8264
offae04cdbe0_0000_cl_2.wav
8263
other7195ffa60_0000_cl_3.wav
8262
go327289eb0_0000_cl_1.wav
8261
other0f3f64d52_0000_cl_3.wav
8260
other9acd02541_0000_cl_3.wav
8259
otherc37a72d30_0000_cl_3.wav
8258
offcb8f83070_0000_cl_2.wav
8257
off924c1a040_0000_cl_2.wav
8256
godc75148d1_0000_cl_1.wav
8255
other743edf9d2_0000_cl_3.wav
8254
otherbf90a57a0_0000_cl_3.wav
8253
other7c1d85334_0000_cl_3.wav
8252
go99081f4d0_0000_cl_1.wav
8251
offc205e6250_0000_cl_2.wav
8250
otherfb24c8261_0000_cl_3.wav
8249
off43691f670_0000_cl_2.wav
8248
other1338a7992_0000_cl_3.wav
8247
go781022050_0000_cl_1.wav
8246
gocb802c631_0000_cl_1.wav
8245
otheref2578c01_0000_cl_3.wav
8244
goeb0676ec0_0000

7982
othercce17a610_0000_cl_3.wav
7981
other4fe019971_0000_cl_3.wav
7980
other9637f43f1_0000_cl_3.wav
7979
other38c30a4a3_0000_cl_3.wav
7978
othereb0676ec3_0000_cl_3.wav
7977
otherf5e44d341_0000_cl_3.wav
7976
offced835d30_0000_cl_2.wav
7975
other00b014452_0000_cl_3.wav
7974
go4c13fe250_0000_cl_1.wav
7973
other2335770e1_0000_cl_3.wav
7972
other66041c690_0000_cl_3.wav
7971
gocd911ace1_0000_cl_1.wav
7970
othere0322f2c0_0000_cl_3.wav
7969
offc634a1890_0000_cl_2.wav
7968
off8281a2a80_0000_cl_2.wav
7967
go475650882_0000_cl_1.wav
7966
godb9cd41d0_0000_cl_1.wav
7965
other3b8406c01_0000_cl_3.wav
7964
other173ae7931_0000_cl_3.wav
7963
otherfd395b741_0000_cl_3.wav
7962
otherf264e0df2_0000_cl_3.wav
7961
go520b2c170_0000_cl_1.wav
7960
off1ba20be20_0000_cl_2.wav
7959
otherdb79a7641_0000_cl_3.wav
7958
offd5ca80c62_0000_cl_2.wav
7957
goc245d3d74_0000_cl_1.wav
7956
other76b587051_0000_cl_3.wav
7955
off32a19ecf1_0000_cl_2.wav
7954
other87014d402_0000_cl_3.wav
7953
off551e42e80_0000_cl_2.wav
7952
otherb0

7663
gob5aacf2c0_0000_cl_1.wav
7662
other377e916b1_0000_cl_3.wav
7661
other3c6d55b30_0000_cl_3.wav
7660
other4def68db1_0000_cl_3.wav
7659
other1dce06e81_0000_cl_3.wav
7658
othercd911ace0_0000_cl_3.wav
7657
off37e8db820_0000_cl_2.wav
7656
other9e42ae250_0000_cl_3.wav
7655
otherc1d39ce86_0000_cl_3.wav
7654
other37b03ab12_0000_cl_3.wav
7653
off953fe1ad1_0000_cl_2.wav
7652
off6d0ad7640_0000_cl_2.wav
7651
goe5dadd240_0000_cl_1.wav
7650
offdc75148d2_0000_cl_2.wav
7649
otherb06c19b01_0000_cl_3.wav
7648
go3cfc6b3a0_0000_cl_1.wav
7647
off88053e921_0000_cl_2.wav
7646
other36746d7f1_0000_cl_3.wav
7645
otherc6a23ff50_0000_cl_3.wav
7644
other93f30cc40_0000_cl_3.wav
7643
go0132a06d3_0000_cl_1.wav
7642
other31d689572_0000_cl_3.wav
7641
other5525ff661_0000_cl_3.wav
7640
goa0f939430_0000_cl_1.wav
7639
off656e8bb30_0000_cl_2.wav
7638
go8c888bbb0_0000_cl_1.wav
7637
other1cc80e390_0000_cl_3.wav
7636
go89d6d11e0_0000_cl_1.wav
7635
other43691f672_0000_cl_3.wav
7634
go0ab3b47d1_0000_cl_1.wav
7633
god55aa56c2

otherf00180d01_0000_cl_3.wav
7265
other39025b210_0000_cl_3.wav
7264
other41dce6f51_0000_cl_3.wav
7263
go686d030b2_0000_cl_1.wav
7262
otherb0bad4a91_0000_cl_3.wav
7261
othered0720f10_0000_cl_3.wav
7260
other0c2d2ffa0_0000_cl_3.wav
7259
go32a19ecf1_0000_cl_1.wav
7258
otherec21c46b0_0000_cl_3.wav
7257
off042186b81_0000_cl_2.wav
7256
go2589f3610_0000_cl_1.wav
7255
otherb59354102_0000_cl_3.wav
7254
go5ff3f9a10_0000_cl_1.wav
7253
offdb9d52d20_0000_cl_2.wav
7252
off9acd02541_0000_cl_2.wav
7251
off39a126483_0000_cl_2.wav
7250
other73cccfaf1_0000_cl_3.wav
7249
other6cf5459b2_0000_cl_3.wav
7248
other56f8a1002_0000_cl_3.wav
7247
other10ace7eb2_0000_cl_3.wav
7246
other2d82a5563_0000_cl_3.wav
7245
other8a56f36e1_0000_cl_3.wav
7244
offe54a0f161_0000_cl_2.wav
7243
god9d6559e0_0000_cl_1.wav
7242
other531a5b8a2_0000_cl_3.wav
7241
go6727b5792_0000_cl_1.wav
7240
goad1429cf1_0000_cl_1.wav
7239
off2796ac500_0000_cl_2.wav
7238
otherde08912f0_0000_cl_3.wav
7237
gocd3331a60_0000_cl_1.wav
7236
other7f74626f4_0

...

2282
go72d75d960_0000_cl_1.wav
2281
other6e8b9b7b2_0000_cl_3.wav
2280
otherfd32732a0_0000_cl_3.wav
2279
other25040e851_0000_cl_3.wav
2278
otherd486fb841_0000_cl_3.wav
2277
other531a5b8a0_0000_cl_3.wav
2276
other8d9cc2601_0000_cl_3.wav
2275
off781022050_0000_cl_2.wav
2274
other3a70ab7f2_0000_cl_3.wav
2273
other6a861f211_0000_cl_3.wav
2272
off3bc211612_0000_cl_2.wav
2271
other518588b60_0000_cl_3.wav
2270
offf4cae1730_0000_cl_2.wav
2269
other0cd323ec2_0000_cl_3.wav
2268
other6c968bd93_0000_cl_3.wav
2267
other0b09edd32_0000_cl_3.wav
2266
otherb9db6e423_0000_cl_3.wav
2265
other26e573a91_0000_cl_3.wav
2264
otherd98f60432_0000_cl_3.wav
2263
off9bea2ac70_0000_cl_2.wav
2262
offe32ff49d2_0000_cl_2.wav
2261
other9a69672b4_0000_cl_3.wav
2260
otherda584bc02_0000_cl_3.wav
2259
goac4b3fc30_0000_cl_1.wav
2258
off6c429c7b4_0000_cl_2.wav
2257
other283d7a530_0000_cl_3.wav
2256
go6e2ef3d51_0000_cl_1.wav
2255
go324210dd1_0000_cl_1.wav
2254
other18c54a681_0000_cl_3.wav
2253
go15d83b540_0000_cl_1.wav
2252
ot

off9a4d12fd2_0000_cl_2.wav
1973
offdf1d50243_0000_cl_2.wav
1972
goe9901cf00_0000_cl_1.wav
1971
other6ceeb9aa2_0000_cl_3.wav
1970
offc6ca5d010_0000_cl_2.wav
1969
goc50225fa0_0000_cl_1.wav
1968
off6a700f9d1_0000_cl_2.wav
1967
off8625475c0_0000_cl_2.wav
1966
other99a9b0151_0000_cl_3.wav
1965
go179a61b72_0000_cl_1.wav
1964
otherb59fe16d1_0000_cl_3.wav
1963
off73af0c500_0000_cl_2.wav
1962
gocc6ee39b3_0000_cl_1.wav
1961
god55aa56c0_0000_cl_1.wav
1960
offadb45ef20_0000_cl_2.wav
1959
offb59354100_0000_cl_2.wav
1958
othera879a2c30_0000_cl_3.wav
1957
other54412eae2_0000_cl_3.wav
1956
gof953e1af4_0000_cl_1.wav
1955
goc245d3d71_0000_cl_1.wav
1954
other24ad3ebe3_0000_cl_3.wav
1953
other7ea032f30_0000_cl_3.wav
1952
go6c968bd93_0000_cl_1.wav
1951
otherb1f8326d3_0000_cl_3.wav
1950
other35d1b6ee1_0000_cl_3.wav
1949
off201741400_0000_cl_2.wav
1948
other6f3428263_0000_cl_3.wav
1947
other414748171_0000_cl_3.wav
1946
go2dce72b30_0000_cl_1.wav
1945
othera591c2ea0_0000_cl_3.wav
1944
go87728a881_0000_cl_1.wav

go3ca784ec0_0000_cl_1.wav
1678
other75b631ff2_0000_cl_3.wav
1677
otherbd8412df2_0000_cl_3.wav
1676
goc120e80e1_0000_cl_1.wav
1675
other9080f6d31_0000_cl_3.wav
1674
other1bb574f91_0000_cl_3.wav
1673
other8f4c551f4_0000_cl_3.wav
1672
other638548d52_0000_cl_3.wav
1671
off553f1a790_0000_cl_2.wav
1670
goffd2ba2f3_0000_cl_1.wav
1669
other5c39594f0_0000_cl_3.wav
1668
other0a7c2a8d2_0000_cl_3.wav
1667
otherd37e4bf11_0000_cl_3.wav
1666
off6205088b0_0000_cl_2.wav
1665
otherc2bc44891_0000_cl_3.wav
1664
other9aa5439d3_0000_cl_3.wav
1663
otheracfd3bc31_0000_cl_3.wav
1662
other161fcca80_0000_cl_3.wav
1661
off51055bda4_0000_cl_2.wav
1660
other918a24730_0000_cl_3.wav
1659
go7f74626f4_0000_cl_1.wav
1658
othera60a09cf1_0000_cl_3.wav
1657
go39543cfd1_0000_cl_1.wav
1656
gob959cd0c1_0000_cl_1.wav
1655
offec21c46b2_0000_cl_2.wav
1654
go439c84f44_0000_cl_1.wav
1653
other1cc80e391_0000_cl_3.wav
1652
go439c84f42_0000_cl_1.wav
1651
other692a88e64_0000_cl_3.wav
1650
offd7529c0c2_0000_cl_2.wav
1649
off617de2213_0

other147754810_0000_cl_3.wav
1310
othera97017df0_0000_cl_3.wav
1309
go8e05039f2_0000_cl_1.wav
1308
other325a0c392_0000_cl_3.wav
1307
go8830e17f0_0000_cl_1.wav
1306
off0132a06d2_0000_cl_2.wav
1305
go5fadb5382_0000_cl_1.wav
1304
offe48a80ed0_0000_cl_2.wav
1303
other01d22d031_0000_cl_3.wav
1302
gofc28c8d80_0000_cl_1.wav
1301
otherbab364201_0000_cl_3.wav
1300
other01b4757a0_0000_cl_3.wav
1299
go1fd85ee41_0000_cl_1.wav
1298
othere2008f391_0000_cl_3.wav
1297
go36050ef33_0000_cl_1.wav
1296
offf8f60f594_0000_cl_2.wav
1295
other05b2db801_0000_cl_3.wav
1294
go71bf8f480_0000_cl_1.wav
1293
go462e19190_0000_cl_1.wav
1292
goc2d15ea51_0000_cl_1.wav
1291
off8a194ee61_0000_cl_2.wav
1290
other8dd788d80_0000_cl_3.wav
1289
go988e2f9a1_0000_cl_1.wav
1288
otherd874a7861_0000_cl_3.wav
1287
otherc84f9b5c0_0000_cl_3.wav
1286
god31d8dd70_0000_cl_1.wav
1285
offe96a50200_0000_cl_2.wav
1284
other96c670732_0000_cl_3.wav
1283
other8a3257491_0000_cl_3.wav
1282
go173ce2be0_0000_cl_1.wav
1281
otheraeb99b1c1_0000_cl_3.w

other413997c11_0000_cl_3.wav
935
goc1d39ce84_0000_cl_1.wav
934
off37a3cd330_0000_cl_2.wav
933
otherbdee441c0_0000_cl_3.wav
932
off56eb74ae2_0000_cl_2.wav
931
otherbfaf20000_0000_cl_3.wav
930
gof1b35ace0_0000_cl_1.wav
929
otherc22d3f180_0000_cl_3.wav
928
otherff21fb591_0000_cl_3.wav
927
offe3b642170_0000_cl_2.wav
926
other48e8b82a2_0000_cl_3.wav
925
other92b0a7352_0000_cl_3.wav
924
off3847619e0_0000_cl_2.wav
923
otherbfdb98011_0000_cl_3.wav
922
go76b587050_0000_cl_1.wav
921
other15c563d70_0000_cl_3.wav
920
othere82914c01_0000_cl_3.wav
919
other9a7c1f834_0000_cl_3.wav
918
other96c66ab70_0000_cl_3.wav
917
otherad1429cf0_0000_cl_3.wav
916
other30802c5d0_0000_cl_3.wav
915
otherd069e6ff0_0000_cl_3.wav
914
goccf418a50_0000_cl_1.wav
913
other6794a7930_0000_cl_3.wav
912
off126a31d21_0000_cl_2.wav
911
other105a0eea0_0000_cl_3.wav
910
goa2b161130_0000_cl_1.wav
909
go7d5f87c80_0000_cl_1.wav
908
other4249c8331_0000_cl_3.wav
907
other6a2fb9a51_0000_cl_3.wav
906
otherb97c9f770_0000_cl_3.wav
905
goe9a